<a href="https://colab.research.google.com/github/KhumairahErnas/Tugas4_JST/blob/main/Tugas4_Khumairah_JST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Import Library yang diperlukan
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 2. Load dataset Iris
iris = load_iris ()
X = iris['data'] # Fitur: sepal length, sepal width, petal length, petal width
y = iris['target'].reshape(-1, 1) # Target: Kelas (Setosa, Versicolor, Virginica)

In [ ]:
# 3. Preprocessing data
# One hot encoding untuk target (karena kita punya 3 kelas)
encoder = OneHotEncoder (sparse_output=False) # sparse_output=False untuk sklearn versi terbaru
y_encoded = encoder.fit_transform(y)

# split data info training and testing sets
X_train, X_test, y_train, y_test = train_test_split (X, y_encoded, test_size=0.2, random_state=42)

# Tampilkan beberapa data latih
train_data = pd.DataFrame (X_train, columns=iris ['feature_names'])
train_data['target'] = np.argmax(y_train, axis=1) # Tambahkan kolom target
train_data['target_name'] = train_data['target'].apply(lambda x:
iris['target_names'] [x]) # Tambahkan nama target
print("Data Latih:")
print(train_data.head())  # Tampilkan 5 data pertama

# Tampilkan beberapa data uji
test_data = pd.DataFrame (X_test, columns=iris['feature_names'])
test_data['target'] = np.argmax(y_test, axis=1) # Tambahkan kolom target
test_data['target_name'] = test_data['target'].apply(lambda x:
iris['target_names'] [x]) # Tambahkan nama target
print("\nData Uji:")
print(test_data.head())   # Tampilkan 5 data pertama

Data Latih:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                4.6               3.6                1.0               0.2   
1                5.7               4.4                1.5               0.4   
2                6.7               3.1                4.4               1.4   
3                4.8               3.4                1.6               0.2   
4                4.4               3.2                1.3               0.2   

   target target_name  
0       0      setosa  
1       0      setosa  
2       1  versicolor  
3       0      setosa  
4       0      setosa  

Data Uji:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                6.1               2.8                4.7               1.2   
1                5.7               3.8                1.7               0.3   
2                7.7               2.6                6.9               2.3   
3                6.0               2.9    

In [ ]:
# 4. Membangun jaringan neural sederhana (1 Hidden Layer)
input_neurons = X_train.shape[1]    # 4 fitur (input layer)
hidden_neurons = 5                  # 5 neuron di hidden layer
output_neurons = y_train.shape[1]   # 3 output (kelas bunga)

# Bobot random inisialisasi
np.random.seed(42)
W1 = np.random.randn(input_neurons, hidden_neurons)
b1 = np.zeros((1, hidden_neurons))
W2 = np.random.randn(hidden_neurons, output_neurons)
b2 = np.zeros((1, output_neurons))

# Fungsi aktivasi : Sigmoid
def sigmoid(x) :
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x) :
  return x * (1 - x)

In [ ]:
# 5. Forward propagation
def forward_propagation(X):
  Z1 = np.dot(X, W1) + b1
  A1 = sigmoid(Z1)
  Z2 = np.dot(A1, W2) + b2
  A2 = sigmoid(Z2)
  return Z1, A1, Z2, A2

In [ ]:
# 6. Backward propagation (perbarui bobot)
def backward_propagation(X, y, Z1, A1, Z2, A2, learning_rate):
  global W1, b1, W2, b2 # Deklarasi variabel global sebelum digunakan

  # Error di output
  dZ2 = A2 - y
  dW2 = np.dot(A1.T, dZ2)
  db2 = np.sum(dZ2, axis=0, keepdims=True)

  # Error di hidden layer
  dA1 = np.dot(dZ2, W2.T)
  dZ1 = dA1 * sigmoid_derivative(A1)
  dW1 = np.dot(X.T, dZ1)
  db1 = np.sum(dZ1, axis=0, keepdims=True)

  # Update bobot
  W1 -= learning_rate * dW1
  b1 -= learning_rate * db1
  W2 -= learning_rate * dW2
  b2 -= learning_rate * db2

In [ ]:
# 7. Training (pelatihan)
def train(X, y, iterations, learning_rate):
  for i in range(iterations):
    Z1, A1, Z2, A2 = forward_propagation(X)
    backward_propagation(X, y, Z1, A1, Z2, A2, learning_rate)

    # Cetak loss (Mean Squared Error) setiap iterasi
    loss = np.mean(np.square(y - A2))
    print(F'Iteration {i+1}, Loss: {loss}')

    # Melatih jaringan (10 iterasi, learning rate 0.01)
    train(X_train, y_train, iterations=10, learning_rate=0.01)

In [ ]:
# 8. Testing (pengujian)
def predict (X):
  _, _, _, A2 = forward_propagation(X)
  return np.argmax(A2, axis=1)

  # Memprediksi kelas untuk data uji
  predictions = predict (X_test)

  # Evaluasi akurasi
  y_test_labels = np.argmax(y_test, axis=1)
  accuracy = accuracy_score (y_test_labels, predictions)
  print(f'Akurasi: {accuracy:.2f}%')

  # Confusion Matrix
  conf_matrix = confusion_matrix(y_test_labels, predictions)
  plt.figure(figsize=(6,4))
  sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
              xticklabels=iris['target_names'], yticklabels=iris['target_names'])
  plt.xlabel ("Predicted")
  plt.ylabel ("Actual")
  plt.title ("Confusion Matrix")
  plt.show()

In [ ]:
# 9. Fungsi untuk prediksi data baru
def predict_new_data(new_data) :
  new_data = np.array(new_data).reshape(1, -1)  # Ubah menjadi array 2D
  predicted_class = predict(new_data) # Prediksi kelas
  class_names = iris['target_names']  # Nama kelas (Setosa, Versicolor, Virginica)
  return class_names[predicted_class] [0]

  # Contoh prediksi data baru
  new_data = [5.1, 3.5, 1.4, 0.2] # Data baru (sepal length, sepal width, petal length, petal width)
  predicted_class = predict_new_data(new_data)
  print(f'Data baru: {new_data}, Prediksi Kelas: {predicted_class}')